# CPU
* 不要用 CPU train DL
* xgboost 等請限制 thread 數量，或改用 GPU
* jobs/worker/process 不要開太多(開了以後看一下 top 別持續高於800%)

# HDD/SSD
大型檔案請放在HDD  
要跑的再搬到SSD(注意下大小，別把SSD塞爆)  

# GPU
* 指定 GPU 前先確認有沒有別人在用
* 沒有要用請釋放顯存(shutdown)
* 正常關閉 subprocess ，避免 GPU 無法正常釋放  
* 盡量提高 GPU 計算力使用率(減少 load data 的等待時間)
* 指定要使用的 GPU ID
```python
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5,6"
```
或
```
CUDA_VISIBLE_DEVICES=5,6 python3 run.py
```
* 不須要整顆顯存的話，可以拿少一點

```python
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
set_session(tf.Session(config=config))
```

# IO
* 請檢查是否有非必要的大量 IO (重複讀取/過度保存)
    * 例如 epoch 超短，又每個 epoch 都存 model

# multi thread/process
因為 CPython 的 multithread 有 GIL  
所以實際上只會有一個 thread 在做計算  
無法增加計算力

* multithread
    * 成本較低
    * 較安全
    * 不會增加算力
    * thread 間溝通較簡單
    * 適用於 IO 密集（開檔案等）
* multiprocess
    * 成本較高
    * 相對危險
    * 可以增加算力
    * process 間溝通較麻煩
    * 適用於 CPU 密集（大量即時計算、查找）


# 常用指令
* nvidia-smi  顯示 GPU 狀況(計算力/顯存使用率等)  
* ps -u -p {pid} 查 preocess 是誰的  
* du -h {dir} 查詢 dir 下的空間使用狀況  
* kill {pid}  強制結束 process  
* top  
* htop  
  

# jupyter notebook

## 由process id 找對應的notebook
ps 或 htop 等找到對應的 command  
如 /usr/bin/python3 -m ipykernel_launcher -f /run/user/1002/jupyter/kernel-54399708-5c8b-4aa4-bc8e-6b7c84a  
取 54399708-5c8b-4aa4-bc8e-6b7c84a (id) 的部分  

到 xxx.xxx.xx.xx:xxxx/api/sessions  
找到對應 id 的 session  
path 即為對應的 notebook  

## shutdown
原理跟直接點 shutdown 一樣
```
%%javascript
Jupyter.notebook.session.delete();
```

# package

## no moudle name xxxxxx
* 檢查是python2還python3
* 是否為環境問題
* 檢查sys.path(可能被蓋掉了，如anaconda)
    * 可於 ~/.bashrc 加入 export PYTHONPATH=/path/to/packages
    * conda env 中開 jupyter, 可能無法正常 import 在env conda install 的 package, 可在 env 中 conda install jupyter 解決


## import matplotlib.pyplot error
可能是沒有對應的 backend  
換個試試 （再沒有就乖乖裝 backend 吧）

```python
import matplotlib  
matplotlib.use('Agg')  
import matplotlib.pyplot as plt  
```

# SSH

## 太久沒通訊斷線
* putty：可設定 Connection -> seconds between keepalives 為 300
* 一般SSH：ServerAliveInterval = 300

## 避免 SSH 斷掉，程式跑到一半掛掉
用 screen/tmux 之類的吧  
別每次都新開啊啊啊啊

### screen
* screen 進入
* screen -r 恢復
* ctrl+a d 離開

## display
```bash
ssh -X catding@theta1
```
如有錯誤，可先用
```bash
xeyes
```
檢查是 display 本身的問題，還是程式有問題

* window:裝 Xming
* mac:直接執行指令，不行的話裝 Xquartz
* linux:直接執行指令

# image

## channel order
* RGB
    * plt
    * PIL.Image
* BGR
    * cv2

## change channel order
```python
# 超快
img[:, :, ::-1]

# 慢
cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
```

## read img to numpy array
cv2 最快

## jpeg
cv2 和 PIL/plt 的解壓算法疑似不同  
會得到不同結果



# time series feature extraction
* tsfresh
    * 會 extrac 出 1400 多個 feature
    * 內部用 pandas 實現，所以有點慢
    * 一段 time series 切太長會炸掉 memory

# Youtube

## download youtube video
推薦 pytube，不過改版的有點頻繁  
小心 doc 內容不一定是最新下載版的（doc會超前...）

## YouTube Data API v3

### quota
每專案每日1,000,000，16：00回覆  
quota 消耗可由此查詢 https://developers.google.com/youtube/v3/determine_quota_cost
* search 100/次
* video info 1~16/次 （視選擇的部分而定）

### api example
* https://www.googleapis.com/youtube/v3/videos?part=snippet&id={videoId}&key={key}
* https://www.googleapis.com/youtube/v3/search?part=snippet&q={key_word}&maxResults=25&order=viewCount&type=video&key={key}

# nvidia-docker

## 開啟
```bash
NV_GPU=6,7 nvidia-docker run -p 6688:8888 gcr.io/tensorflow/tensorflow:1.3.0-gpu-py3
```

## Got permission denied while trying to connect to the Docker daemon ......
要加入 docker group